In [1]:
import numpy as np
import pandas as pd
import pyNetLogo
import os
import ipyparallel
# Start IPCluster with 8 engines now through the command line using the following command:
# IPCluster start -n 8
# 8 engines are used for fair comparison against 8 parallel headless workspaces through NL4Py
client = ipyparallel.Client()
print(client.ids)
direct_view = client[:]
import os
#Push the current working directory of the notebook to a "cwd" variable on the engines that can be accessed later
direct_view.push(dict(cwd=os.getcwd()))

[0, 1, 2, 3, 4, 5, 6, 7]


<AsyncResult: _push>

In [2]:
%%px #Jupyter notebook decorator to make the notebook cell parallelizable
import os
os.chdir(cwd) # change to the current directory to the IPCluster engines
import pyNetLogo
import pandas as pd
# Replace argument netlogo_home with the location of your NetLogo installation 
netlogo = pyNetLogo.NetLogoLink(gui=False, netlogo_home = "C:\Program Files\NetLogo 6.0.3", netlogo_version = '6')
netlogo.load_model('Wolf Sheep Predation.nlogo')

In [3]:
def simulation(runId):
    # Random initialization of parameters 
    # Same netlogo commands as used for the NL4Py evaluation
    netlogo.command("set initial-number-sheep random-float 250")
    netlogo.command("set initial-number-wolves random-float 250")
    netlogo.command("set grass-regrowth-time random-float 100")
    netlogo.command("set sheep-gain-from-food random-float 50")
    netlogo.command("set wolf-gain-from-food random 100")
    netlogo.command("set sheep-reproduce random-float 20")
    netlogo.command("set wolf-reproduce random-float 20")
    netlogo.command("set show-energy? false")
    netlogo.command('set model-version "sheep-wolves-grass"')
    netlogo.command('setup')
    #Run for 100 ticks and return the number of sheep and wolf agents at each time step
    counts = netlogo.repeat_report(['ticks','count sheep','count wolves'], 100)    
    
    results = pd.Series([counts.shape[0],
                         counts.iloc[-1:1], 
                         counts.iloc[-1:2]],index=['ticks','Avg. sheep', 'Avg. wolves'])
    
    return results

In [4]:
import time
for j in range (0,10):
    startTime = int(round(time.time() * 1000))
    # Create and pass in a load balanced view for the IPCluster engines
    lview = client.load_balanced_view()
    # Run the simulations and measure execution time
    results = pd.DataFrame(lview.map_sync(simulation, range(5000)))
    stopTime = int(round(time.time() * 1000))
    print(stopTime - startTime)
    startTime = int(round(time.time() * 1000))
    lview = client.load_balanced_view()
    results = pd.DataFrame(lview.map_sync(simulation, range(10000)))
    stopTime = int(round(time.time() * 1000))
    print(stopTime - startTime)
    startTime = int(round(time.time() * 1000))
    lview = client.load_balanced_view()
    results = pd.DataFrame(lview.map_sync(simulation, range(15000)))
    stopTime = int(round(time.time() * 1000))
    print(stopTime - startTime)

276841
546403
807366
267383
542267
805883
268950
539585
807710
279567
537670
811613
269288
542348
795602
269109
537350
803900
269971
530641
811763
263795
548901
807123
271505
530246
805531
261356
538444
814524
